In [2]:
import os

os.chdir("..")

In [59]:
from pathlib import Path

## Torchtext

In [34]:
from torchtext.data import functional as F
import torchtext.transforms as T

import torch

In [108]:
SPE_FILE = "./data/processed/tglang_spe.txt"
MODE = "word"
SIZE = 10000

WRITE_TO = f"./models/trained/tglang_spe_{MODE}_{SIZE}"

In [109]:
F.generate_sp_model(SPE_FILE, vocab_size=SIZE, model_type=MODE, model_prefix=WRITE_TO)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=./data/processed/tglang_spe.txt --model_prefix=./models/trained/tglang_spe_word_10000 --vocab_size=10000 --model_type=word
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./data/processed/tglang_spe.txt
  input_format: 
  model_prefix: ./models/trained/tglang_spe_word_10000
  model_type: WORD
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  p

In [110]:
spe = T.SentencePieceTokenizer(WRITE_TO + ".model")

In [111]:
CODE = """import React from 'react';

class MyComponent extends React.Component {
  constructor(props) {
    super(props);
    this.state = {
      count: 0
    };
  }

  handleClick = () => {
    this.setState(prevState => ({
      count: prevState.count + 1
    }));
  }

  render() {
    return (
      <div>
        <h1>Counter: {this.state.count}</h1>
        <button onClick={this.handleClick}>Increment</button>
      </div>
    );
  }
}

export default MyComponent;
"""

In [112]:
spe(CODE)

['▁import',
 '▁React',
 '▁from',
 "▁'react';",
 '▁class',
 '▁MyComponent',
 '▁extends',
 '▁React.Component',
 '▁{',
 '▁constructor(props)',
 '▁{',
 '▁super(props);▁this.state',
 '▁=',
 '▁{',
 '▁count:',
 '▁0',
 '▁};',
 '▁}',
 '▁handleClick',
 '▁=',
 '▁()',
 '▁=>',
 '▁{',
 '▁this.setState(prevState',
 '▁=>',
 '▁({',
 '▁count:',
 '▁prevState.count',
 '▁+',
 '▁1',
 '▁}));',
 '▁}',
 '▁render()',
 '▁{',
 '▁return',
 '▁(',
 '▁<div>',
 '▁<h1>Counter:▁{this.state.count}</h1>',
 '▁<button',
 '▁onClick={this.handleClick}>Increment</button>',
 '▁</div>',
 '▁);',
 '▁}',
 '▁}',
 '▁export',
 '▁default',
 '▁MyComponent;']

In [113]:
scripted = torch.jit.script(spe)

assert scripted(CODE) == spe(CODE)

In [114]:
torch.jit.save(scripted, WRITE_TO + ".torchscript")

## Tokenizers

In [70]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer.pre_tokenizer = Whitespace()

tokenizer.train([SPE_FILE], trainer)

In [71]:
tokenizer.save("./models/trained/tokenizer.json")